In [2]:
import functions as fct
import pandas as pd
import dynamics as dyn
import numpy as np
import dask
from cluster import new_launch_cluster
import graphics as gx

# parameter setting
nb_banks=50
path = "./results/parameter_space/test/"
initial_deposits_size = 40

# reg ratios
alpha_init=False  # initial cash (< 1/(1-gamma) - beta)
alpha=0.01
beta_init=0.5 # initial collateral  (< 1/(1-gamma) - alpha)
beta_reg=0.5
beta_star=0.5
gamma=0.03
gamma_init = 2.5*gamma
collateral_value=1.0

# initialisation of deposits size
initialization_method="pareto"
alpha_pareto=1.3

# shocks on deposits 
shocks_method="non-conservative"
shocks_law="normal-mean-reverting"
shocks_vol=0.01 # 0.05

# min trans size
min_repo_trans_size=1e-8  # 1e-8

# dynamics & ploting
nb_steps=4*int(5e3)
dump_period=int(5e3)
plot_period=int(5e2)
cp_option=True
heavy_plot=False

# LCR mgt
LCR_mgt_opt=True

# leverage mgt
end_repo_period=False # if int, periodic end repo / if false, leverage mgt
gamma_star = 1.5*gamma

# money creation
loan_tenor=nb_steps # if int, money creation / if false. no new loans
loan_period=1
new_loans_vol = 0 # standard deviation around the mean creation of loans
new_loans_mean = 5e-2/250 # daily mean increase in loans expressed as a percentage of the intital loans (the other option would be a relative increase but it is unstable np.exp(np.log(1.01)/250)-1 = 4e-5)
beta_new = beta_reg # if number, new colat / if false, no new colat 
# gamma_new = gamma_star*(beta_reg+1)/(1+gamma_star*beta_reg) # if number, new own funds / if false, no new own funds. we need to also add beta reg to ensure LCR can be covered with additional cash (and we still match the solvency ratio despite this excess of stress) in the case of no payment shocks and no collateral creation
gamma_new = gamma_init

# substitution of collateral
substitution = False

dic_default_value = {
    "nb_banks": nb_banks,
    "initial_deposits_size": initial_deposits_size,
    "alpha_init": alpha_init,
    "alpha": alpha,
    "beta_init": beta_init,
    "beta_reg": beta_reg,
    "beta_star": beta_star,
    "beta_new": beta_new,
    "gamma_init": gamma_init,
    "gamma": gamma,
    "gamma_star": gamma_star, 
    "gamma_new": gamma_new, 
    "collateral_value": collateral_value,
    "initialization_method": initialization_method,
    "alpha_pareto": alpha_pareto,
    "shocks_method": shocks_method,
    "shocks_law": shocks_law,
    "shocks_vol": shocks_vol,
    "LCR_mgt_opt": LCR_mgt_opt,
    "min_repo_trans_size": min_repo_trans_size,
    "loan_tenor": loan_tenor, # for money creation 
    "loan_period": loan_period,
    "new_loans_vol": new_loans_vol,
    "new_loans_mean": new_loans_mean,
    "end_repo_period": end_repo_period,
    "nb_steps": nb_steps,
    "path_results": f"{path}runs/",
    "dump_period": dump_period,
    "plot_period": plot_period,
    "cp_option": cp_option,
    "heavy_plot": heavy_plot,  # False to avoid the number of linux node to explode
    "substitution":substitution,
}

# special dictionary of parameters ranges 
list_dic_range_special = [
    # {"gamma_init": [k*gamma for k in np.arange(2,3,0.1)],
    # "gamma_new": [k*gamma for k in np.arange(2,3,0.1)],},
    {"min_repo_trans_size": np.logspace(-15, 5, num=100)},
]

# general dictionary of parameters ranges
list_dic_range_general = [
    {"nb_banks": np.arange(1, 3)},
    {"initial_deposits_size": np.logspace(-9, 3, num=50)},
    {"alpha_init": np.arange(0, 1, 0.1)},
    {"beta_init": np.arange(0, 1, 0.05)},
    {"beta_reg": np.arange(0, 1, 0.05)},
    {"alpha_pareto": np.logspace(0, 1, num=3)},
    {"shocks_vol": np.arange(0, 0.30, 0.0025)},
    {"min_repo_trans_size": np.logspace(-16, 2, num=3)},
    {"gamma_star": np.arange(0, 10*gamma, 0.01)},
] 

# define the dictionary to be used for the ranges
list_dic_range = list_dic_range_special


# run parameter space analysis

In [3]:
440/60

7.333333333333333

In [4]:
# initialize the path
# fct.delete_n_init_path(path)

# build list of the dic_args to be tested
list_dic_args = fct.build_args(dic_default_value, list_dic_range)

# open a cluster
client, cluster = new_launch_cluster(
    task_memory=19,
    job_walltime="12:00:00",
    max_cpu=len(list_dic_args),
)

# run with dask distributed
dld_obj = [
    dask.delayed(dyn.single_run)(**dic_args) for dic_args in list_dic_args
]
futures = client.compute(dld_obj)


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.70.211.9:8787/status,
Dashboard: http://10.70.211.9:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.70.211.9:42305,Workers: 0
Dashboard: http://10.70.211.9:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
! squeue -u vlecoz -h -t pending,running -r -O "state" | uniq -c

In [5]:
for i, future in enumerate(futures):
    if future.status == "finished":
        print(i, future.result())

1 None
2 None
4 None
5 None
7 None
8 None
9 None
10 None
11 None
13 None
14 None
15 None
16 None
18 None
19 None
21 None
22 None
23 None
26 None
27 None
29 None
30 None
31 None
33 None
34 None
35 None
36 None
38 None
39 None
40 None
41 None
42 None
43 None
44 ***ERROR***: leverage ratio not above its regulatory level for one or several banks. Plot and stop simulation.
45 None
46 None
47 None
50 None
51 None
53 None
54 None
55 None
56 None
57 None
58 None
59 None
60 None
61 None
63 None
64 None
65 None
66 None
68 None
69 None
70 None
71 None
72 None
74 None
75 None
76 None
77 None
78 None
79 None
80 None
81 None
82 None
83 None
84 None
85 None
86 None
87 None
88 None
89 None
90 None
91 None
92 None
93 None
94 None
95 None
96 None
97 None
98 None
99 None


In [8]:
future

<Future: lost, type: NoneType, key: single_run-ab4f7b86-9aeb-4a13-ab36-f9aa87fdf2e6>

In [6]:
client.shutdown()

In [ ]:
dyn.single_run(*dic_default_value.values())

# collect results

In [6]:
# collect results into df_network_sensitivity
df_network_sensitivity = fct.get_df_network_sensitivity(dic_default_value["path_results"])

100%|██████████| 3/3 [01:22<00:00, 27.64s/it]


# plot results

In [ ]:
import parameters as par
par.df_plt.loc["gamma_new", "convertion"]

In [2]:
# plot the sensitivity
df_network_sensitivity = pd.read_csv(f"{dic_default_value['path_results']}df_network_sensitivity.csv", index_col=(0,1))
gx.plot_all_sensitivities(df_network_sensitivity,path=path)

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:14<00:00,  1.23it/s]


In [4]:
gamma_init*100

9.0

In [15]:
df_network_sensitivity.loc["gamma_new"]["collateral reuse"]

0.015                   0.016647
0.018                   0.041164
0.020999999999999998    0.040801
0.0225                  0.033127
0.023999999999999997         NaN
0.026999999999999996         NaN
0.027                   0.081112
0.029999999999999995    0.115062
0.0315                  0.132621
0.032999999999999995    0.150487
0.03599999999999999          NaN
0.036                   0.195930
0.03899999999999999          NaN
0.040499999999999994    0.254340
0.041999999999999996         NaN
0.04499999999999999          NaN
0.04799999999999999          NaN
0.049499999999999995    0.429458
0.05099999999999999     0.437255
0.053999999999999986         NaN
0.05399999999999999     0.504381
0.05699999999999999     0.552280
0.05849999999999999     0.583236
0.059999999999999984    0.604268
0.06299999999999999     0.654519
0.063                   0.668460
0.06599999999999999     0.692433
0.06899999999999999          NaN
0.07199999999999998     0.791605
0.07499999999999998     0.874123
0.07799999